<a href="https://colab.research.google.com/github/rcleveng/notebooks/blob/main/Ask_Questions_about_'The_Wealth_of_Nations'_using_RAG_with_Google_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple RAG langchain example using Google's Generate AI API.
This example let's you ask questions against a book.

You'll need an API key from Generative AI Studio.


In [ ]:
!pip install kaleido python-multipart tensorflow-probability fastapi uvicorn milvus[client] pymilvus langchain langchain-cli "langserve[all]" google.generativeai langchain-google-genai


Restart the kernel once dependencies are updated

In [18]:
quit()

Start Milvus vector database that is used for holding the vectors.

In [ ]:
import pyarrow
print(pyarrow.__version__)

import pymilvus
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)
print("Using pymilvus version: ", pymilvus.__version__)

from milvus import default_server, debug_server

# Cleanup anything from the default server
if default_server.running:
  print("stopping the default server")
  default_server.stop()
print("Cleaning up old files from the default server")
default_server.cleanup()

# Cleanup anything from the debug server
if debug_server.running:
  print("stopping the debug server")
  debug_server.stop()
  connections.disconnect('default')
print("Cleaning up old files from the debug server")
debug_server.cleanup()

# Start your milvus debug server
print("Starting the debug server")
debug_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
try:
 connections.connect(host='127.0.0.1', port=debug_server.listen_port)
except Exception as e:
  print(e)
print("Connections: ", connections.list_connections())

# Check if the server is ready.
print("Started milvus server version ", utility.get_server_version())



In [3]:
# @title Set the book URL to use for Q&A

# Change this to use a different book.
BOOK_URL="https://www.gutenberg.org/cache/epub/3300/pg3300.txt" #@param {type:"string"}

In [4]:
# @title Fetch and split URL into chunks

import requests
from google.colab import userdata

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document

# Get the key from https://makersuite.google.com
# Add your PALM_KEY as a secret to colab.

google_api_key = userdata.get('PALM_KEY')

# Get text from URL
def get_data(url):
 r = requests.get(url)
 return r.text

BOOK_TEXT = get_data(BOOK_URL)
doc =  Document(page_content=BOOK_TEXT, metadata={"source": "local"})
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents([doc])

# Count the number of chunks
print ('Finished splitting document into ', len(texts), 'chunks.')


Finished loaded chunks:  3102 total.


In [5]:
# @title Load chunks into vector database

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from langchain.vectorstores import Milvus


# Create collection

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key, task_type="retrieval_document")


vector_db = Milvus.from_documents(
   texts,
   embeddings,
   connection_args={"host": "127.0.0.1", "port": default_server.listen_port},
)


In [25]:
# @title Ask Questions on The Wealth of Nations

# Create the retriever
from langchain.globals import (
    set_debug,
    set_verbose
)

debug_enabled = False #@param {type:"boolean"}
verbose_enabled = True #@param {type:"boolean"}
set_debug(debug_enabled)
set_verbose(verbose_enabled)

model_name = "models/text-bison-001" # @param ["gemini-pro", "models/text-bison-001"] {allow-input: false}

llm = GoogleGenerativeAI(model=model_name, google_api_key=google_api_key)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_db.as_retriever())

# Ask question
query = "What are the benefits of specialization and the division of labor?" #@param {type:"string"}

qa.invoke({"query": query})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

or class of philosophers; and this subdivision of employment in
      philosophy, as well as in every other business, improves dexterity, and
      saves time. Each individual becomes more expert in his own peculiar
      branch, more work is done upon the whole, and the quantity of science is
      considerably increased by it.
    
      It is the great multiplication of the productions of all the different
      arts, in consequence of the division of labour, which occasions, in a
      well-governed society, that universal opulence which extends itself to the
      lowest ranks of the people. Every workman has a great quantity of his own
      work to dispose of beyond what he hi

{'query': 'What are the benefits of specialization and the division of labor?',
 'result': 'The benefits of specialization and the division of labor are: increased dexterity, saves time, more work is done upon the whole, and the quantity of science is considerably increased.'}

In [16]:
# OPTIONAL: Stop your milvus server
default_server.stop()
debug_server.stop()